# Docker replication

In the recent experiment we found out, that it is possible to reproduce results on one laptop:
    
- with disabled GPU
- with 5 points stated further


However, it was not possible to reproduce results across the computers/systems. We have checked the randomization in train-test split, intialization of the weigths, tried Adagrad (deterministic) and confirmed, that with the same parameters and fixed seed the training process yeilds different training weights and accuracy results across two platforms(that are however in the same region, ~10**-3) 

With GPU it is not possible to get reproducible results, see keras documentation on
[How can I obtain reproducible results using Keras during development?](https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development)


Battling non-reproducibility at the code lvl is shown [here](https://wandb.ai/sayakpaul/reproducible-ml/reports/Reproducible-Models-with-W-B--Vmlldzo3ODMxNQ)

### Random seeds + fixed (initial) weights

There are two obvious solutions here -

Fix the random number generator seed so that each time you try to generate random numbers, you get the exact same results. I suggest doing this for all the libraries (that support fixing seeds) you're using in your experiment. In NumPy and TensorFlow, it can be done like so -

```python
SEED = 666
  import tensorflow as tf
  tf.random.set_seed(SEED)
  import numpy as np
  np.random.seed(SEED)
  ```

It should be also noted that this code should stay at the top of your code i.e. before you start any experimentation.

Another solution would be to serialize the weights of the network before you start to train it. That way, we would have access to the initial weights of the network, and we can use them to help us produce the same results every time we train the network (with the same configuration).

Now, of course, there are other parts of a neural network that can introduce non-determinism – **dropout layers, sampling layers (remember VAEs?), latent vectors to name a few**. The non-determinism they introduce is of the good kind, as it often helps neural networks perform better. We can run our training data through these layers a number of times and measure the average deviation in each of the results. If there isn't anything wrong, the deviations won't be very high.

In [4]:
! pip install --upgrade wandb
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: wandb in /home/test_user/.local/lib/python3.7/site-packages (0.10.18)
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 394.3 MB 6.1 kB/s  eta 0:00:01  |                                | 788 kB 4.8 MB/s eta 0:01:22     |▍                               | 4.4 MB 4.8 MB/s eta 0:01:21     |▍                               | 4.5 MB 4.8 MB/s eta 0:01:21     |▍                               | 5.0 MB 4.8 MB/s eta 0:01:21     |▍                               | 5.4 MB 4.8 MB/s eta 0:01:21     |▌                               | 6.1 MB 4.8 MB/s eta 0:01:21     |▌                               | 6.6 MB 4.8 MB/s eta 0:01:21     |▋                               | 7.2 MB 3.6 MB/s eta 0:01:47     |█▏                              | 14.0 MB 3.6 MB/s eta 0:01:45     |█▎                              | 15.5 MB 5.7 MB/s eta 0:01:07   

In [5]:
import wandb

# initialize wandb run
#wandb.init(mode='offline')
wandb.init(project='cnn')

Set environment variable PYTHONHASHSEED in terminal - didn't work in Jupyter otherwise

$ export PYTHONHASHSEED=0

In [25]:
seed_value=0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
%env 'PYTHONHASHSEED'=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from tensorflow.compat.v1.keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

env: 'PYTHONHASHSEED'=str(seed_value)


In [26]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import sys

In [27]:
# Model configuration
img_width, img_height = 28, 28
no_classes = 10
verbosity = 1
batch_size = 250
no_epochs = 1
validation_split = 0.2
convKernelSize = 3
activation = 'relu'
dropout = 0.25
class_weight = None
epsilon = 1e-07
initial_epoch=0
learning_rate=0.001
max_queue_size=10
#num_layers = 0 can't be set in guild.ai?
optimizer_name = 'Adam'
sample_weight=None
shuffle=True
steps_per_epoch = None
use_multiprocessing=False
valdiation_batch_size=None
validation_freq=1
validation_split=0.2
validation_steps=None
workers=2

wandb.config = {
    'img_width' : 28,
    'img_height' : 28,
    'no_classes' : 10,
    'verbosity' : 1,
    'batch_size' : 250,
    'no_epochs' : 1,
    'validation_split' : 0.2,
    'convKernelSize' : 3,
    'activation' : 'relu',
    'dropout' : 0.25,
    'class_weight' : None,
    'epsilon' : 1e-07,
    'initial_epoch':0,
    'learning_rate':0.001,
    'max_queue_size':10,
    #num_layers = 0 can't be set in guild.ai?
    'optimizer_name' : 'Adam',
    'sample_weight':None,
    'shuffle':True,
    'steps_per_epoch' : None,
    'use_multiprocessing':False,
    'valdiation_batch_size':None,
    'validation_freq':1,
    'validation_split':0.2,
    'validation_steps':None,
    'workers':2
}

In [28]:
print(wandb.config)

{'img_width': 28, 'img_height': 28, 'no_classes': 10, 'verbosity': 1, 'batch_size': 250, 'no_epochs': 1, 'validation_split': 0.2, 'convKernelSize': 3, 'activation': 'relu', 'dropout': 0.25, 'class_weight': None, 'epsilon': 1e-07, 'initial_epoch': 0, 'learning_rate': 0.001, 'max_queue_size': 10, 'optimizer_name': 'Adam', 'sample_weight': None, 'shuffle': True, 'steps_per_epoch': None, 'use_multiprocessing': False, 'valdiation_batch_size': None, 'validation_freq': 1, 'validation_steps': None, 'workers': 2}


In [29]:
###wandb.config.update({'img_height':'28'})

In [30]:
# Load MNIST dataset
# deterministic with np.seed
(input_train, target_train), (input_test, target_test) = mnist.load_data()
target_train, target_test
wandb.config.update({'input_train':input_train, 'target_train':target_train, 'input_test':input_test, 'target_test':target_test})

In [31]:
wandb.config

{'img_width': 28,
 'img_height': 28,
 'no_classes': 10,
 'verbosity': 1,
 'batch_size': 250,
 'no_epochs': 1,
 'validation_split': 0.2,
 'convKernelSize': 3,
 'activation': 'relu',
 'dropout': 0.25,
 'class_weight': None,
 'epsilon': 1e-07,
 'initial_epoch': 0,
 'learning_rate': 0.001,
 'max_queue_size': 10,
 'optimizer_name': 'Adam',
 'sample_weight': None,
 'shuffle': True,
 'steps_per_epoch': None,
 'use_multiprocessing': False,
 'valdiation_batch_size': None,
 'validation_freq': 1,
 'validation_steps': None,
 'workers': 2,
 'input_train': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[

(array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [32]:
#Reshape data 
input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1) 

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

wandb.config.update({'input_train_reshape':input_train, 'target_train_reshape':target_train, 'input_test_reshape':input_test, 'target_test_reshape':target_test})

# Convert into [0, 1] range.
input_train = input_train / 255
input_test = input_test / 255

# Convert target vectors to categorical targets
target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

wandb.config.update({'input_train_normalized':input_train, 'target_train_categorical':target_train, 'input_test_normalized':input_test, 'target_test_categorical':target_test})

In [33]:
# Create the model
# put seed to all layers but MaxPooling and Flatten
model = Sequential()
model.add(Conv2D(32, kernel_size=(convKernelSize, convKernelSize), activation=activation, input_shape=input_shape, kernel_initializer=tf.keras.initializers.glorot_normal(seed=seed_value)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout, seed=seed_value))
model.add(Conv2D(64, kernel_size=(convKernelSize, convKernelSize), activation=activation, kernel_initializer=tf.keras.initializers.glorot_normal(seed=seed_value)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout, seed=seed_value))
model.add(Flatten())
model.add(Dense(256, activation=activation, kernel_initializer=tf.keras.initializers.glorot_normal(seed=seed_value)))
model.add(Dense(no_classes, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal(seed=seed_value)))

In [34]:
weights, biases = model.layers[0].get_weights()
# weights are the same with initialization
weights

array([[[[-0.15831769, -0.04490802, -0.06178062,  0.00811734,
          -0.02479651,  0.0084381 , -0.07956775,  0.06733285,
           0.15519604, -0.01092447,  0.03560017,  0.04341984,
          -0.05477583, -0.00358999, -0.05533827,  0.0323185 ,
           0.08563322,  0.02418045, -0.10292246, -0.12518083,
           0.09223624, -0.03020439,  0.12570627,  0.0975105 ,
          -0.11563888,  0.10041784,  0.10081172,  0.0287824 ,
           0.00732475,  0.05259866, -0.04275294,  0.06155232]],

        [[ 0.0789836 , -0.12635088, -0.01206782, -0.12652083,
          -0.02337128, -0.01320411, -0.11903066,  0.03582355,
           0.08965569,  0.0286646 , -0.03971971, -0.03905942,
          -0.17780222, -0.04269768,  0.13343556,  0.02158995,
          -0.05882326,  0.05944131, -0.02419369, -0.12406069,
           0.01600565,  0.07905839,  0.13416708,  0.06617373,
          -0.10046838, -0.05296212,  0.10336909, -0.05892661,
          -0.00624058, -0.13513325,  0.08659808, -0.1850473 ]],

  

In [35]:
# Compile the model and use the Adagrad
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(), #Adam
              metrics=['accuracy'])

In [36]:
# Fit data to model
from wandb.keras import WandbCallback

# Add labels
labels =["0","1","2","3","4","5","6","7","8","9"]

"""monitor='val_loss',
              verbose=0,
              mode='auto',
              save_weights_only=False,
              log_weights=False,
              log_gradients=False,
              save_model=True,
              training_data=None,
              validation_data=None,
              labels=[],
              data_type=None,
              predictions=36,
              generator=None,
              input_type=None,
              output_type=None,
              log_evaluation=False,
              validation_steps=None,
              class_colors=None,
              log_batch_frequency=None,
              log_best_prefix='best_'"""

history = model.fit(input_train, target_train,
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              validation_split=validation_split,
                   callbacks=[WandbCallback(data_type="image", labels=labels)])

192/192 [==============================] - 46s 234ms/step - loss: 0.7419 - accuracy: 0.7687 - val_loss: 0.0946 - val_accuracy: 0.9728


wandb: WARNING No validation_data set, pass a generator to the callback.


In [37]:
# Generate generalization metrics
#sys.stdout = open('/dev/stdout', 'w')
score = model.evaluate(input_test, target_test, verbose=0)
print(f'TestLoss: {score[0]}')
print(f'TestAccuracy: {score[1]}')
print('ValLoss: ',history.history['val_loss'][-1])
print('ValAccuracy: ',history.history['val_accuracy'][-1])
#print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

TestLoss: 0.08609030395746231
TestAccuracy: 0.972599983215332
ValLoss:  0.09458783268928528
ValAccuracy:  0.9727500081062317


In [38]:
wandb.config.update({'model_summary':model.summary})

## Run#1: docker MacOS

TestLoss: 0.08609030395746231

TestAccuracy: 0.972599983215332

ValLoss:  0.09458783268928528

ValAccuracy:  0.9727500081062317

## Run #2: docker MacOS

TestLoss: 0.08609030395746231

TestAccuracy: 0.972599983215332

ValLoss:  0.09458783268928528

ValAccuracy:  0.9727500081062317

In [39]:
weights, biases = model.layers[0].get_weights()

In [40]:
weights

array([[[[-1.37746811e-01, -8.21383223e-02, -9.64649916e-02,
           3.90559807e-02, -3.31303850e-02,  1.91143379e-02,
          -9.37926024e-02,  7.41249695e-02,  2.07462221e-01,
          -8.63095671e-02,  1.40347287e-01,  3.25050056e-02,
          -1.15937501e-01,  4.88959737e-02, -4.16884534e-02,
           8.84919539e-02,  8.10976699e-02,  1.09264314e-01,
          -1.03267156e-01, -1.32670045e-01,  1.12038560e-01,
          -2.84015946e-02,  1.60024866e-01,  1.33542046e-01,
          -1.48786575e-01,  9.70320180e-02,  1.28268778e-01,
          -3.21602374e-02,  1.58859249e-02,  2.14563590e-02,
           2.99491314e-03,  3.92571688e-02]],

        [[ 1.16007753e-01, -9.89379957e-02,  3.51074687e-03,
          -1.10032432e-01, -1.68555174e-02,  1.31477676e-02,
          -1.81516320e-01,  5.01150973e-02,  1.49884060e-01,
          -3.32989097e-02,  4.31759655e-02, -4.79170047e-02,
          -1.37366876e-01, -3.93461548e-02,  1.48708314e-01,
           4.85640317e-02, -6.90265819

In [41]:
!printenv

MPLBACKEND=module://ipykernel.pylab.backend_inline
PYTHONHASHSEED=0
HOSTNAME=b45d99256a95
SHLVL=1
HOME=/home/test_user
OLDPWD=/home/test_user
PAGER=cat
TF2_BEHAVIOR=1
_=/opt/conda/bin/python3
TERM=xterm-color
PATH=/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
LANG=C.UTF-8
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dw